In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
df=pd.DataFrame(data=data.data,columns=data.feature_names)
df_target=data.target

In [ ]:
df=pd.DataFrame(data=data.data,columns=data.feature_names)
df_target=data.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df,df_target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=0)

In [ ]:
class Gravity_algo:

  def __init__(self,objective_function,n_iteration=50,population_size=50,g0=100,eps=0.01):
    self.population_size = population_size
    self.n_iteration=n_iteration
    self.objective_function=objective_function
    self.g0=g0
    self.eps=eps


  def initialize_population(self,X):
        
        self.individuals =  np.random.randint(0,2,size=(self.population_size,X.shape[1]))


  def evaluate_fitness(self,model,X_train,y_train,X_valid,y_valid):
        scores =  []
        for i,individual in enumerate(self.individuals):
            chosen_features = [index for index in range(X_train.shape[1]) if individual[index]==1]
            X_train_copy = X_train.iloc[:,chosen_features]
            X_valid_copy = X_valid.iloc[:,chosen_features]
            score = self.objective_function(model,X_train_copy,y_train,X_valid_copy,y_valid)
            if score< self.current_best_scores:
              self.current_best_scores=score
              self.current_best_score_dimensions=individual
            scores.append(score)
        self.fitness_scores = scores
 
  def sigmoid(self,x):
    return 1/(1+np.exp(-x))
        
  def fit(self,model,X_train,y_train,X_valid,y_valid):
    self.initialize_population(X_train)
    self.velocities=np.zeros((self.population_size,X_train.shape[1]))
    self.current_best_scores=np.inf
    self.current_best_score_dimensions=np.random.randint(0,2,size=(X_train.shape[1]))

    for iteration in range(self.n_iteration):
      self.evaluate_fitness(model,X_train,y_train,X_valid,y_valid)
      self.gi=self.g0*(np.exp(-20*iteration/self.n_iteration))
      self.fitness_scores_numpy=np.array(self.fitness_scores)
      self.qi=np.array(self.fitness_scores_numpy-self.fitness_scores_numpy.max())/(self.fitness_scores_numpy.min()-self.fitness_scores_numpy.max())
      self.Mi=self.qi/self.qi.sum()
      #int(np.tan(np.pi-np.arctan((self.population_size-1)/self.n_iteration))*iteration+self.population_size)
      best_iteration_individuals=self.individuals[np.argsort(self.fitness_scores)[: 5] ]
      best_iteration_individuals_masses=self.Mi[np.argsort(self.fitness_scores)[:5 ]]
      self.interim_acc=np.zeros((self.population_size,X_train.shape[1]))
      for single_individual,single_individual_mass in zip(best_iteration_individuals,best_iteration_individuals_masses):
        self.interim_acc+=np.random.random()*(self.individuals-single_individual)*( self.gi*single_individual_mass )* np.repeat( ( 1/( ((self.individuals-single_individual)**2).sum(axis=1)+self.eps ) ),X_train.shape[1]).reshape(self.population_size,X_train.shape[1])
        #self.interim_acc+=(self.individuals-single_individual)*( self.gi*single_individual_mass )* np.repeat( ( 1/( ((self.individuals-single_individual)**2).sum(axis=1)+self.eps ) ),X_train.shape[1]).reshape(self.population_size,X_train.shape[1])
      
      
      self.velocities+=self.interim_acc+self.velocities*np.random.random((self.population_size,X_train.shape[1]))
      self.velocities=np.where(self.velocities>6,6,self.velocities)
      self.velocities=np.where(self.velocities<-6,-6,self.velocities)
      self.individuals=np.where(np.random.uniform(size=(self.population_size,X_train.shape[1]))<=np.tanh(self.velocities),1-self.individuals,self.individuals)

      print('iteration ',iteration,' ',np.array(self.fitness_scores).min(),' |mean ',np.array(self.fitness_scores).mean(),' |best_score ',self.current_best_scores)
     

    return self.current_best_score_dimensions


In [ ]:
import scipy
from sklearn.metrics import log_loss
def score_model(model,X_train, y_train, X_valid, y_valid):
    model.fit(X_train,y_train)
    return log_loss(y_valid,model.predict_proba(X_valid))
gaf=Gravity_algo(score_model,100,30,100)
res=gaf.fit(clf,X_train, y_train,X_test, y_test)